# 📘 2D Gaussian Fitting: Image Analysis and Peak Detection⏱️ **20-30 minutes** | 📊 **Level: ●●○ Intermediate** | 🏷️ **Core Tutorial**---

## 🎯 What You'll Learn- ✓ Fit 2D Gaussian functions to image data- ✓ Handle rotation and elliptical shapes- ✓ Manage memory for 2D datasets- ✓ Visualize 2D fitting results---

## Setup and ConfigurationConfigure environment for GPU memory handling:

In [ ]:
import osimport sysos.environ["JAX_PREALLOCATE_GPU_MEMORY"] = "false"os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"import jax.numpy as jnpfrom nlsq import CurveFit, estimate_memory_requirementsprint("✅ Environment configured")

## Define 2D Gaussian ModelModel: $f(x,y) = A \cdot e^{-0.5(X^2/\sigma_x^2 + Y^2/\sigma_y^2)} + C$Where X, Y are rotated coordinates.

In [ ]:
def rotate_coordinates2D(coords, theta):    """Rotate 2D coordinates by angle theta."""    R = jnp.array([[jnp.cos(theta), -jnp.sin(theta)],                    [jnp.sin(theta), jnp.cos(theta)]])    shape = coords[0].shape    coords = jnp.stack([coord.flatten() for coord in coords])    rcoords = R @ coords    return [jnp.reshape(coord, shape) for coord in rcoords]def gaussian2d(coords, n0, x0, y0, sigma_x, sigma_y, theta, offset):    """2D Gaussian with rotation."""    coords = [coords[0] - x0, coords[1] - y0]    X, Y = rotate_coordinates2D(coords, theta)    density = n0 * jnp.exp(-0.5 * (X**2 / sigma_x**2 + Y**2 / sigma_y**2))    return density + offsetprint("✅ 2D Gaussian model defined")

## Generate Synthetic 2D DataCreate test image with Gaussian peak:

In [ ]:
import numpy as npimport matplotlib.pyplot as plt# Create meshgridlength = 200x = np.linspace(-5, 5, length)y = np.linspace(-5, 5, length)X, Y = np.meshgrid(x, y)# True parametersparams_true = [100, 0, 0, 1.5, 1.0, np.pi/4, 10]# Generate datadata_true = gaussian2d([X, Y], *params_true)noise = np.random.normal(0, 5, data_true.shape)data = data_true + noiseplt.figure(figsize=(10, 4))plt.subplot(121)plt.imshow(data_true, cmap="viridis")plt.title("True 2D Gaussian")plt.subplot(122)plt.imshow(data, cmap="viridis")plt.title("With Noise")plt.tight_layout()plt.show()print(f"Data shape: {data.shape}")

## Fit the 2D GaussianFit with parameter bounds:

In [ ]:
# Initial guessp0 = [90, 0.5, 0.5, 1.2, 0.8, 0, 5]# Boundsbounds = ([0, -5, -5, 0.1, 0.1, -np.pi, 0],          [200, 5, 5, 5, 5, np.pi, 50])# Fitjcf = CurveFit()popt, pcov = jcf.curve_fit(    gaussian2d,    [X, Y],    data,    p0=p0,    bounds=bounds)print("Fitted Parameters:")print(f"  Amplitude: {popt[0]:.1f} (true: {params_true[0]})")print(f"  Center X:  {popt[1]:.2f} (true: {params_true[1]})")print(f"  Center Y:  {popt[2]:.2f} (true: {params_true[2]})")print(f"  Sigma X:   {popt[3]:.2f} (true: {params_true[3]})")print(f"  Sigma Y:   {popt[4]:.2f} (true: {params_true[4]})")print(f"  Rotation:  {popt[5]:.2f} rad (true: {params_true[5]:.2f})")print(f"  Offset:    {popt[6]:.1f} (true: {params_true[6]})")

## Visualize ResultsCompare data, fit, and residuals:

In [ ]:
# Generate fitted surfacedata_fit = gaussian2d([X, Y], *popt)residuals = data - data_fitplt.figure(figsize=(15, 4))plt.subplot(131)plt.imshow(data, cmap="viridis")plt.title("Data")plt.colorbar()plt.subplot(132)plt.imshow(data_fit, cmap="viridis")plt.title("Fit")plt.colorbar()plt.subplot(133)plt.imshow(residuals, cmap="RdBu", vmin=-15, vmax=15)plt.title("Residuals")plt.colorbar()plt.tight_layout()plt.show()rmse = np.sqrt(np.mean(residuals**2))print(f"\nRMSE: {rmse:.2f}")

## Key Takeaways1. **2D Gaussian fitting** handles image analysis and peak detection2. **Rotation parameter** enables elliptical peak fitting3. **Memory management** is crucial for large images4. **Bounds** ensure physical constraints are met---

## 🔗 Next Steps**Continue learning:**- [Large Dataset Demo](large_dataset_demo.ipynb)- [Performance Optimization](performance_optimization_demo.ipynb)- [Advanced Topics](../03_advanced/)---